## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [Clinical De-identification for Spanish](https://aws.amazon.com/marketplace/pp/prodview-otbdi7i5fwrsa).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Clinical Deidentification Spanish

Deidentification is essential for safeguarding patient privacy in clinical data, including texts, PDFs, images, and DICOM files containing Protected Health Information (PHI). PHI encompasses various health-related data, including common identifiers such as name, address, birth date, and Social Security Number.

- **Model**: [es.deid.clinical](https://nlp.johnsnowlabs.com/2023/06/17/clinical_deidentification_es.html)
- **Model Description**: This pipeline is trained with sciwiki_300d embeddings and can be used to deidentify PHI information from medical texts in Spanish. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask, fake or obfuscate the following entities: AGE, DATE, PROFESSION, E-MAIL, USERNAME, LOCATION, DOCTOR, HOSPITAL, PATIENT, URL, IP, MEDICALRECORD, IDNUM, ORGANIZATION, PHONE, ZIP, ACCOUNT, SSN, PLATE, SEX and IPADDR

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [7]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse

In [8]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [9]:
model_name = "es-deid-clinical"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [10]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

### Important Parameter

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    Example: "**Nombre: Danilo. Apellidos: Ramos Portillo. NHC: 4376372. NASS: 35 61051794 56. Domicilio: Calle Fernando Higueras, 27, 7B.**"

    - **masked**: Default policy that masks entities with their type.

      -> 'Nombre: `<PATIENT>`. Apellidos: `<PATIENT>`. NHC: `<SSN>`. NASS: `<ID>`. Domicilio: `<PATIENT>`, `<AGE>`, 7B.'

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

      -> 'Nombre: `Aurora Garrido Paez`.Apellidos: `Antonio González Cuevas`.NHC: `BBBBBBBBQR648597`.NASS: `48.124.111S`. Domicilio: `Aurora Garrido Paez`, `36`, 7B.'

    - **masked_fixed_length_chars**: Masks entities with a fixed length of asterisks (\*).

      -> 'Nombre: `****`. Apellidos: `****`. NHC: `****`. NASS: `****`. Domicilio: `****`, `****`, 7B.'

    - **masked_with_chars**: Masks entities with asterisks (\*).

      -> 'Nombre: [`****`]. Apellidos: [`************`]. NHC: [`*****`]. NASS: [`************`]. Domicilio: [`*********************`], `**`, 7B.'
    
You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked"
}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [12]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [13]:
docs = [
''' Nombre: David. Apellidos: Flores Blanco. NHC: 8972604. Domicilio:  Calle Puerto Rico, 17, 3 C. Loca... 
 Nombre: David. Apellidos: Flores Blanco. NHC: 8972604. Domicilio:  Calle Puerto Rico, 17, 3 C. Localidad/ Provincia: Albacete. CP: 02541. Datos asistenciales. Fecha de nacimiento: 20/10/2003. País: España. Edad: 13 años Sexo: H. Fecha de Ingreso: 04/01/2017. Médico: María Flores Copete NºCol: 02 02 12630.
Informe clínico del paciente: Niño de 13 años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o prurito. No había recibido ningún tratamiento, ni refería ningún factor desencadenante. En la exploración se observó una placa, indurada a la palpación, con un área discrómica central y un halo violáceo, de distribución lineal en el tercio medio-distal en la zona externa del muslo derecho. No presentaba otras lesiones cutáneas, ni clínica sistémica acompañante.
Padre con rasgos de Asperger no diagnosticado.
Se solicitó una analítica que incluía hemograma, bioquímica, anticuerpos antinucleares (ANA) y serologías, examen de orina, así como biopsia de la lesión. Los resultados fueron normales, con ANA, anti-Sm, anti-RNP, anti-SSA, anti-SSB, anti-Jo1 y anti-Scl70 negativos. La serología para Borrelia burgdorferi fue negativa y el análisis de orina no mostró anomalías. La histopatología fue compatible con el diagnóstico de morfea.
Ante una morfea lineal en la infancia sin afectación articular ni deformidad estética, se decidió iniciar tratamiento corticoideo tópico con propionato de fluticasona, una aplicación al día durante 2 meses, seguido de calcipotriol tópico, también un aplicación diaria. Tras casi un año de seguimiento, las lesiones se han estabilizado.
Remitido por: Dra. María Flores Copete. Centro de Salud Zona 4. C/ Seminario, 4. C.P. 02006. Albacete. España. Correo electrónico: florescopetemaria@gmail.com''',

'''Nombre: Alexia. Apellidos: Hernandez Peña. NHC: 5347636/301. Domicilio: Av. Sta. María 281, Recoleta... 
Nombre: Alexia. Apellidos: Hernandez Peña. NHC: 5347636/301. Domicilio: Av. Sta. María 281, Recoleta, Región Metropolitana. Localidad/ Provincia: Santiago.
CP: 8420000. Datos asistenciales. Fecha de nacimiento: 21/06/1971. País: Chile. Edad: 44 años Sexo: Mujer. Fecha de Ingreso: 09/08/2015. Médico: Julio Villanueva Maffei.
Motivo de ingreso: Paciente mujer, 44 años, derivado a nuestra unidad por aumento de volumen en la región cigomática y limitación progresiva de la apertura bucal de 10 meses de evolución. La historia médica no era relevante. 
Exploración: El examen físico extraoral muestra asimetría facial por aumento de volumen en la mejilla izquierda, de limites difusos, consistencia ósea, indoloro, con piel de aspecto normal. La apertura bucal era 30 mm. La articulación temporomandibular (ATM) era normal a la palpación, sin ruidos o dolor en apertura. En la ortopantomografía observamos una apófisis coronoides izquierda de mayor tamaño. La hipótesis diagnóstica fue tumor coronoídeo.
Pruebas complementarias y evolución:  Se realizó una incisión por sobre el borde anterior de la rama mandibular, desinsertando todas las inserciones del músculo temporal. Alrededor del aumento de volumen se encontró una pseudocápsula fibrosa, que fue liberada. Se efectuó la coronoidectomía. La apertura bucal se recuperó inmediatamente hasta 43 mm. El postoperatorio se desarrolló sin complicaciones, y la paciente fue dada de alta a las 48 horas.
El estudio histopatológico informó la presencia fibras, hueso de neoformación y tejido cartilaginoso hialino. El diagnóstico fue osteocondroma. Los controles posteriores se desarrollaron sin problemas. El alta definitiva fue dada a los 10 meses, con remodelación casi completa de la deformidad de la mejilla.
Remitido por: Dr. Julio Villanueva Maffei. Santa María 571, Recoleta Chile E-mail: javm@vtr.net'''

]


sample_text = """Nombre: Marina . Apellidos: Casas Gómez. NHC: 8455635. Domicilio: Calle del Dr. Esquerdo 99, Bajo A ... 
Nombre: Marina . Apellidos: Casas Gómez. NHC: 8455635. Domicilio: Calle del Dr. Esquerdo 99, Bajo A . Localidad/ Provincia: Madrid.
CP: 28007. Datos asistenciales. Fecha de nacimiento: 21/07/2007. País: España. Edad: 7 años Sexo: M. Fecha de Ingreso: 03/11/2014. Servicio/ Unidad: Infectología. Episodio: 524652564. Médico: Miguel Peñafiel Freire  NºCol: 28 28 97102. Historia Actual: Niña de siete años nacida en España y sin patología previa que es remitida a la Unidad de Infectología Pediátrica por adenopatías dolorosas submandibulares izquierdas de un mes de evolución, sin respuesta al tratamiento con amoxicilina-ácido clavulánico y antiinflamatorio oral.
La paciente no había presentado fiebre, sudoración nocturna ni pérdida de peso. No estaba vacunada de BCG. Durante un mes al año visita Marruecos, donde tiene contacto con animales (ovejas, corderos) y consume lácteos sin pasteurizar.
Exploración física: En la exploración física presenta un conglomerado submandibular izquierdo de adenopatías, doloroso a la palpación, duro, no rodadero ni adherido, la mayor de las adenopatías de 2 × 2 cm. El resto de la exploración física y anamnesis por aparatos no presentaba hallazgos patológicos.
Resumen de pruebas complementarias: Dada la falta respuesta al tratamiento y teniendo en cuenta los datos epidemiológicos, se realizó la prueba de tuberculina, que fue positiva, con una induración de 10 mm con patrón vesicular a las 72 horas. La radiografía de tórax y la ecografía abdominal fueron normales.
Las serologías de Brucella, virus de Epstein-Barr (VEB) y citomegalovirus (CMV) fueron negativas. La analítica mostró hemograma normal, velocidad de sedimentación globular (VSG) de 47 mm/hora y el interferon-gamma release assays (IGRA) (QuantiFERON(r)) fue positivo, con un valor de 10,15 (rango normal: 0-0,35).
Se realizó una punción aspiración con aguja fina (PAAF) de la adenopatía, que mostraba un material espeso de coloración blanquecina. El cultivo y el estudio citológico revelaron un patrón de linfadenitis granulomatosa necrotizante.
Evolución y comentarios: Dada la escasa afectación de la paciente y la sospecha de adenitis tuberculosa, se pautó un tratamiento domiciliario inicial con isoniacida, rifampicina y pirazinamida, a la espera del cultivo de la muestra obtenida mediante PAAF. El cultivo fue positivo para M. bovis, presentando un antibiograma con resistencia a pirazinamida y sensibilidad a estreptomicina, etambutol, isoniacida y rifampicina. Por este motivo se sustituyó la pirazinamida por etambutol durante dos meses, continuando posteriormente con isoniacida y rifampicina hasta finalizar seis meses. Los cultivos de micobacterias en jugos gástricos fueron negativos. Durante ese tiempo la paciente acudió a revisiones periódicas en el Servicio de Infectología Pediátrica, en las que se constató una buena evolución clínica y la resolución completa del proceso infeccioso.
Remitido por:  Dr. Miguel Peñafiel Freire. Hospital General Universitario Gregorio Marañón. Calle del Dr. Esquerdo, 46, 28007 Madrid. Correo electrónico: mpfreire.89@gmail.com"""

### JSON

#### Example 1: masked (default-policy)

In [14]:
input_json_data = {"text": sample_text}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
print(response_json["predictions"][0])

Nombre: <PROFESSION> . Apellidos: <PATIENT>. NHC: <SSN>. Domicilio: Calle del Dr. <DOCTOR>, Bajo A ... 
Nombre: <PROFESSION> . Apellidos: <PATIENT>. NHC: <SSN>. Domicilio: Calle del Dr. <DOCTOR>, Bajo A . Localidad/ Provincia: <LOCATION>.
CP: <ZIP>. Datos asistenciales. Fecha de nacimiento: <DATE>. País: <LOCATION>. Edad: <AGE> años Sexo: <SEX>. Fecha de Ingreso: <DATE>. Servicio/ Unidad: <PROFESSION>. Episodio: <PHONE>. <PROFESSION>: Miguel Peñafiel Freire  NºCol: <USERNAME> <USERNAME> <USERNAME>. Historia <SSN>: Niña de siete años nacida en España y sin patología previa que es remitida a la Unidad de Infectología Pediátrica por adenopatías dolorosas submandibulares izquierdas de un mes de evolución, sin respuesta al tratamiento con amoxicilina-ácido clavulánico y antiinflamatorio oral.
La paciente <SSN> había presentado fiebre, sudoración nocturna ni pérdida de peso. No estaba vacunada de BCG. Durante un mes al año visita <LOCATION>, donde tiene contacto con animales (ovejas, cordero

#### Example 2: obfuscated

In [16]:
input_json_data = {"text": sample_text, "masking_policy": "obfuscated"}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
print(response_json["predictions"][0])

Nombre: Mecánico . Apellidos: Sr. Mulet. NHC: 3144784. Domicilio: Calle del Dr. Perlita Amorós Coronado, Bajo A ... 
Nombre: Mecánico . Apellidos: Sr. Mulet. NHC: 3144784. Domicilio: Calle del Dr. Perlita Amorós Coronado, Bajo A . Localidad/ Provincia: Valencia.
CP: 53990. Datos asistenciales. Fecha de nacimiento: 21/07/2007. País: Barcelona. Edad: 9 años Sexo: F.. Fecha de Ingreso: 03/11/2014. Servicio/ Unidad: Otorrinolaringóloga. Episodio: 451745471. juez: Miguel Peñafiel Freire  NºCol: HO78 HO78 adela79. Historia Xdcfxe: Niña de siete años nacida en España y sin patología previa que es remitida a la Unidad de Infectología Pediátrica por adenopatías dolorosas submandibulares izquierdas de un mes de evolución, sin respuesta al tratamiento con amoxicilina-ácido clavulánico y antiinflamatorio oral.
La paciente kn había presentado fiebre, sudoración nocturna ni pérdida de peso. No estaba vacunada de BCG. Durante un mes al año visita Valencia, donde tiene contacto con animales (ovejas, c

### JSON Lines

In [18]:
def create_jsonl(records, masking_policy=None):
    json_records = []

    if isinstance(records, str):
        records = [records]

    for text in records:
        record = {"text": text}

        if masking_policy is not None:
            record["masking_policy"] = masking_policy
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record, ensure_ascii=False) for record in json_records)
    return json_lines


#### Example 1: masked (default-policy)

In [19]:
input_jsonl_data = create_jsonl(sample_text, masking_policy="masked")
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": "Nombre: <PROFESSION> . Apellidos: <PATIENT>. NHC: <SSN>. Domicilio: Calle del Dr. <DOCTOR>, Bajo A ... \nNombre: <PROFESSION> . Apellidos: <PATIENT>. NHC: <SSN>. Domicilio: Calle del Dr. <DOCTOR>, Bajo A . Localidad/ Provincia: <LOCATION>.\nCP: <ZIP>. Datos asistenciales. Fecha de nacimiento: <DATE>. País: <LOCATION>. Edad: <AGE> años Sexo: <SEX>. Fecha de Ingreso: <DATE>. Servicio/ Unidad: <PROFESSION>. Episodio: <PHONE>. <PROFESSION>: Miguel Peñafiel Freire  NºCol: <USERNAME> <USERNAME> <USERNAME>. Historia <SSN>: Niña de siete años nacida en España y sin patología previa que es remitida a la Unidad de Infectología Pediátrica por adenopatías dolorosas submandibulares izquierdas de un mes de evolución, sin respuesta al tratamiento con amoxicilina-ácido clavulánico y antiinflamatorio oral.\nLa paciente <SSN> había presentado fiebre, sudoración nocturna ni pérdida de peso. No estaba vacunada de BCG. Durante un mes al año visita <LOCATION>, donde tiene contacto con anima

#### Example 2: obfuscated

In [21]:
input_jsonl_data = create_jsonl(sample_text, masking_policy="obfuscated")
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": "Nombre: Mecánico . Apellidos: Sr. Mulet. NHC: 3144784. Domicilio: Calle del Dr. Perlita Amorós Coronado, Bajo A ... \nNombre: Mecánico . Apellidos: Sr. Mulet. NHC: 3144784. Domicilio: Calle del Dr. Perlita Amorós Coronado, Bajo A . Localidad/ Provincia: Valencia.\nCP: 53990. Datos asistenciales. Fecha de nacimiento: 21/07/2007. País: Barcelona. Edad: 9 años Sexo: F.. Fecha de Ingreso: 03/11/2014. Servicio/ Unidad: Otorrinolaringóloga. Episodio: 451745471. juez: Miguel Peñafiel Freire  NºCol: HO78 HO78 adela79. Historia Xdcfxe: Niña de siete años nacida en España y sin patología previa que es remitida a la Unidad de Infectología Pediátrica por adenopatías dolorosas submandibulares izquierdas de un mes de evolución, sin respuesta al tratamiento con amoxicilina-ácido clavulánico y antiinflamatorio oral.\nLa paciente kn había presentado fiebre, sudoración nocturna ni pérdida de peso. No estaba vacunada de BCG. Durante un mes al año visita Valencia, donde tiene contacto con

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [24]:
validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [25]:
# Create JSON and JSON Lines data
input_json_data = {
    "input1.json": json.dumps({"text": docs, "masking_policy": "masked"}, ensure_ascii=False),
    "input2.json": json.dumps({"text": docs, "masking_policy": "obfuscated"}, ensure_ascii=False),
    "input3.json": json.dumps({"text": docs, "masking_policy": "masked_fixed_length_chars"}, ensure_ascii=False),
    "input4.json": json.dumps({"text": docs, "masking_policy": "masked_with_chars"}, ensure_ascii=False),
}

input_jsonl_data = {
    "input1.jsonl": create_jsonl(docs, masking_policy="masked"),
    "input2.jsonl": create_jsonl(docs, masking_policy="obfuscated"),
    "input3.jsonl": create_jsonl(docs, masking_policy="masked_fixed_length_chars"),
    "input4.jsonl": create_jsonl(docs, masking_policy="masked_with_chars")
}

# Upload JSON and JSON Lines data to S3
for file_name, data in input_json_data.items():
    upload_to_s3(data, file_name)

for file_name, data in input_jsonl_data.items():
    upload_to_s3(data, file_name)


### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [27]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [28]:
masking_policies = {
    "masked": "input1.json",
    "obfuscated": "input2.json",
    "masked_fixed_length_chars": "input3.json",
    "masked_with_chars": "input4.json",
}

for policy_name, validation_file_name in masking_policies.items():
    print("-"*50, policy_name ,"-"*50)
    retrieve_json_output_from_s3(validation_file_name)
    print("\n")

-------------------------------------------------- masked --------------------------------------------------


{'predictions': ['Nombre: <PATIENT>. Apellidos: <PATIENT>. NHC: <SSN>. Domicilio:  <LOCATION>, 17, 3 C. Loca... \n Nombre: <PATIENT>. Apellidos: <PATIENT>. NHC: <SSN>. Domicilio:  <LOCATION>, 17, 3 C. Localidad/ Provincia: <LOCATION>. CP: <ZIP>. Datos asistenciales. Fecha de nacimiento: <DATE>. País: <LOCATION>. Edad: <AGE> años Sexo: <SEX>. Fecha de Ingreso: <DATE>. <PROFESSION>: María Flores Copete NºCol: <USERNAME> <USERNAME> <USERNAME>.\nInforme clínico del paciente: <SSN> de <AGE> años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o prurito. No había recibido ningún tratamiento, ni refería ningún factor desencadenante. En la exploración se obse



-------------------------------------------------- obfuscated --------------------------------------------------


{'predictions': ['Nombre: Emigdio Lledó. Apellidos: Emma Tormo Alemán. NHC: 3605791. Domicilio:  Valencia, 17, 3 C. Loca... \n Nombre: Emigdio Lledó. Apellidos: Emma Tormo Alemán. NHC: 3605791. Domicilio:  Valencia, 17, 3 C. Localidad/ Provincia: Bilbao. CP: 95412. Datos asistenciales. Fecha de nacimiento: 20/10/2003. País: Barcelona. Edad: 18 años Sexo: M.. Fecha de Ingreso: 04/01/2017. juez: María Flores Copete NºCol: celia05 celia05 omayo.\nInforme clínico del paciente: Kvnn de 18 años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o prurito. No había recibido ningún tratamiento, ni refería ningún factor desencadenante. En la exploración se observ



-------------------------------------------------- masked_fixed_length_chars --------------------------------------------------


{'predictions': ['Nombre: ****. Apellidos: ****. NHC: ****. Domicilio:  ****, 17, 3 C. Loca... \n Nombre: ****. Apellidos: ****. NHC: ****. Domicilio:  ****, 17, 3 C. Localidad/ Provincia: ****. CP: ****. Datos asistenciales. Fecha de nacimiento: ****. País: ****. Edad: **** años Sexo: ****. Fecha de Ingreso: ****. ****: María Flores Copete NºCol: **** **** ****.\nInforme clínico del paciente: **** de **** años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o prurito. No había recibido ningún tratamiento, ni refería ningún factor desencadenante. En la exploración se observó una placa, indurada a la palpación, con un área discrómica central y un halo 



-------------------------------------------------- masked_with_chars --------------------------------------------------


{'predictions': ['Nombre: [***]. Apellidos: [***********]. NHC: [*****]. Domicilio:  [***************], 17, 3 C. Loca... \n Nombre: [***]. Apellidos: [***********]. NHC: [*****]. Domicilio:  [***************], 17, 3 C. Localidad/ Provincia: [******]. CP: [***]. Datos asistenciales. Fecha de nacimiento: [********]. País: [****]. Edad: ** años Sexo: *. Fecha de Ingreso: [********]. [****]: María Flores Copete NºCol: ** ** [***].\nInforme clínico del paciente: [**] de ** años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o prurito. No había recibido ningún tratamiento, ni refería ningún factor desencadenante. En la exploración se observó una placa, ind

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [30]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [31]:
masking_policies = {
    "masked": "input1.jsonl",
    "obfuscated": "input2.jsonl",
    "masked_fixed_length_chars": "input3.jsonl",
    "masked_with_chars": "input4.jsonl",
}

for policy_name, validation_file_name in masking_policies.items():
    print("-"*50, policy_name ,"-"*50)
    retrieve_jsonlines_output_from_s3(validation_file_name)
    print("\n")

-------------------------------------------------- masked --------------------------------------------------
{"predictions": "Nombre: <PATIENT>. Apellidos: <PATIENT>. NHC: <SSN>. Domicilio:  <LOCATION>, 17, 3 C. Loca... \n Nombre: <PATIENT>. Apellidos: <PATIENT>. NHC: <SSN>. Domicilio:  <LOCATION>, 17, 3 C. Localidad/ Provincia: <LOCATION>. CP: <ZIP>. Datos asistenciales. Fecha de nacimiento: <DATE>. País: <LOCATION>. Edad: <AGE> años Sexo: <SEX>. Fecha de Ingreso: <DATE>. <PROFESSION>: María Flores Copete NºCol: <USERNAME> <USERNAME> <USERNAME>.\nInforme clínico del paciente: <SSN> de <AGE> años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o pruri

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

